In [12]:
'''
File Created: Monday, 12th August 2019 1:19:00 pm
Author: Peng YUN (pyun@ust.hk)
Copyright 2018 - 2019 RAM-Lab, RAM-Lab
'''
import open3d as o3
from open3d import JVisualizer
from IPython.display import clear_output
import warnings
import os
import math
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from PIL import Image, ImageDraw
import numpy as np
from det3.methods.pointnet.model import PointNetCls, feature_transform_regularizer
from det3.methods.pointnet.config import cfg
from det3.methods.pointnet.dataset import ShapeNetDataset

In [13]:
cfg.train_dict["RESUME"] = '/usr/app/det3/methods/pointnet/saved_weights/PointNet-000A/10.pth.tar'

In [14]:
model = PointNetCls(k=cfg.model_dict["K"],
                    feature_transform=cfg.model_dict["FEATURE_TRANSFORM"])
if cfg.task_dict["GPU"] is not None:
    warnings.warn('You have chosen a specific GPU. This will completely '
                  'disable data parallelism.')
    print("Use GPU: {} for training".format(cfg.task_dict["GPU"]))
    torch.cuda.set_device(cfg.task_dict["GPU"])
    model = model.cuda(cfg.task_dict["GPU"])
else:
    model = torch.nn.DataParallel(model).cuda()

if cfg.train_dict["RESUME"]:
    if os.path.isfile(cfg.train_dict["RESUME"]):
        print("=> loading checkpoint '{}'".format(cfg.train_dict["RESUME"]))
        checkpoint = torch.load(cfg.train_dict["RESUME"])
        cfg.train_dict["START_EPOCH"] = checkpoint['epoch']
        best_loss1 = checkpoint['best_loss1']
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
                   .format(cfg.train_dict["RESUME"], checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(cfg.train_dict["RESUME"]))
cudnn.benchmark = True

test_dataset = ShapeNetDataset(
    root=cfg.data_dict["DATA_DIR"],
    classification=True,
    split='test',
    npoints=cfg.data_dict["NUM_POINTS"],
    data_augmentation=False)

cls2ind = {'Airplane': 0, 'Bag': 1, 'Cap': 2,
           'Car': 3, 'Chair': 4, 'Earphone': 5,
           'Guitar': 6, 'Knife': 7, 'Lamp': 8,
           'Laptop': 9, 'Motorbike': 10, 'Mug': 11,
           'Pistol': 12, 'Rocket': 13, 'Skateboard': 14,
           'Table': 15}
ind2cls = dict()
for k,v in cls2ind.items():
    ind2cls[v] = k

Use GPU: 0 for training
=> loading checkpoint '/usr/app/det3/methods/pointnet/saved_weights/PointNet-000A/10.pth.tar'
=> loaded checkpoint '/usr/app/det3/methods/pointnet/saved_weights/PointNet-000A/10.pth.tar' (epoch 11)
{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}
{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  after removing the cwd from sys.path.


In [15]:
def vis(idx):
    model.eval()
    with torch.no_grad():
        points, target = test_dataset[idx]
        points = points.unsqueeze(0)
        points = points.transpose(2, 1)
        if cfg.task_dict["GPU"] is not None:
            points = points.contiguous().cuda(cfg.task_dict["GPU"], non_blocking=True)
            target = target.contiguous().cuda(cfg.task_dict["GPU"], non_blocking=True)
        current_batch_size = points.shape[0]
        pred, _, _ = model(points)
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()

        est = ind2cls[int(pred_choice[0].cpu().numpy())]
        gt = ind2cls[int(target[0].cpu().numpy())]
        print("{}, Target: {}, Est: {}".format(idx, gt, est))
        fragment = o3.PointCloud()
        fragment.points = o3.utility.Vector3dVector(points.transpose(2, 1)[0].cpu().numpy())
        visualizer = JVisualizer()
        visualizer.add_geometry(fragment)
        visualizer.show()
    

In [16]:
vis(0)

0, Target: Airplane, Est: Airplane


JVisualizer with 1 geometries